Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p0">Part 0</a>: Kaggle Competition
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pieplines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

In [1]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
from sklearn.model_selection import RandomizedSearchCV

In [79]:
from sklearn.linear_model import SGDClassifier

In [2]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [3]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english')
rfc = RandomForestClassifier()

In [4]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('clf', rfc)
                ])

In [5]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   38.9s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [6]:
grid_search.best_score_

0.8949824970828472

In [7]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([1, 1], dtype=int64)

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [9]:
import zipfile
with zipfile.ZipFile("data/train.csv.zip","r") as zip_ref:
    zip_ref.extractall("data")

In [96]:
import pandas as pd
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [49]:
train.head()

,id,description,category
0,1,A marriage of 13 and 18 year old bourbons. A m...,2
1,2,There have been some legendary Bowmores from t...,1
2,3,This bottling celebrates master distiller Park...,2
3,4,What impresses me most is how this whisky evol...,1
4,9,"A caramel-laden fruit bouquet, followed by une...",2


In [41]:
# obviously I should add some some words...
import spacy
from spacy.tokenizer import Tokenizer

In [42]:
nlp = spacy.load("en_core_web_lg")

In [43]:
# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

In [44]:
# STOP_WORDS = nlp.Defaults.stop_words.union(['i\'ll', 'it.', '..', 'place', 'don\'t', '-', 'it\'s', 'i\'ve', 'i\'m', 's', ' '])

In [47]:
# tokens = []

# for doc in tokenizer.pipe(train['description'], batch_size=500):
    
#     doc_tokens = []
    
#     for token in doc: 
#         if token.text.lower() not in STOP_WORDS:
#             doc_tokens.append(token.text.lower())
   
#     tokens.append(doc_tokens)
    
# train['tokens'] = tokens

In [51]:
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_ for token in doc if (token.is_stop != True) 
            and (token.is_punct != True)]

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate vectorizer object
# ngram_range 
# since classification I would tune the ngram_range
tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.95, ngram_range=(1,2))

# Create a vocabulary and get word counts per document
# Learn vocab and transform data into form we want
sparse = tfidf.fit_transform(train['description'])

# Print word counts

# Get feature names to use as dataframe column headers
dtm = pd.DataFrame(sparse.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
dtm.head()

,,,£,10,100,12,12 year,add,age,aged,...,white pepper,wine,wood,year,year old,young,youthful,£,’,’s
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.128351,0.158396,0.0,0.0,0.0,0.000000,0.000000
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.147052,0.000000
2,0.0,0.147901,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.294783,0.121262,0.0,0.0,0.0,0.200937,0.117776
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
4,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000


In [97]:
vect = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.95, ngram_range=(1,2))
rfc = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', rfc)])

In [ ]:
parameters = {
    'vect__max_df': ( 0.6, .75),
    'vect__min_df': (.01, .02),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(20, 30),
    'clf__max_depth':(20, 50)
}

rand_search = RandomizedSearchCV(pipe, parameters, cv=5, n_jobs=4, verbose=1)
# grid_search.fit(data.data, data.target)
rand_search.fit(train['description'], train['category'])

### Define Pipeline Components

In [105]:
vect = TfidfVectorizer(stop_words='english')
rfc = RandomForestClassifier()
pipe = Pipeline([('vect', vect), ('clf', rfc)])

In [57]:
vect = TfidfVectorizer(stop_words='english', min_df=0.02, max_df=.90, ngram_range=(1,2))
rfc = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', rfc)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [106]:
parameters = {
    'vect__max_df': ( 0.5, .6),
    'vect__min_df': (.01, .02),
    'vect__max_features': (500, 800),
    'clf__n_estimators':(50, 100),
    'clf__max_depth':(50, 75)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=4, verbose=1)
# grid_search.fit(data.data, data.target)
grid_search.fit(train['description'], train['category'])

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   23.7s
[Parallel(n_jobs=4)]: Done 160 out of 160 | elapsed:  1.6min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'vect__max_df': (0.5, 0.6), 'vect__min_df': (0.01, 0.02), 'vect__max_features': (500, 800), 'clf__n_estimators': (50, 100), 'clf__max_depth': (50, 75)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [28]:
# Nothing custom...
grid_search.best_score_

0.8669760247486465

In [32]:
# can't remember XD
grid_search.best_score_

0.8839907192575406

In [107]:
grid_search.best_score_

0.8928847641144625

In [108]:
grid_search.best_params_

{'clf__max_depth': 50,
 'clf__n_estimators': 100,
 'vect__max_df': 0.5,
 'vect__max_features': 800,
 'vect__min_df': 0.02}

In [65]:
grid_search.best_score_

0.8897911832946636

In [63]:
grid_search.best_params_

{'clf__max_depth': 50,
 'clf__n_estimators': 50,
 'vect__max_df': 0.6,
 'vect__max_features': 800,
 'vect__min_df': 0.02}

In [67]:
grid_search.best_score_

0.8936581593194122

In [68]:
grid_search.best_params_

{'clf__max_depth': 50,
 'clf__n_estimators': 100,
 'vect__max_df': 0.5,
 'vect__max_features': 800,
 'vect__min_df': 0.02}

In [132]:
vect = TfidfVectorizer(stop_words='english', max_features=1000, ngram_range=(1,2), min_df=0.02, max_df=.7)
rfc = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced')
pipe = Pipeline([('vect', vect), ('clf', rfc)])
                             

In [100]:
# vect = TfidfVectorizer(stop_words='english', min_df=0.02, max_df=.5, ngram_range=(1,2), max_features=1000)
# rfc = RandomForestClassifier()#n_estimators=120, random_state=42, n_jobs=-1,
#                          #class_weight='balanced')
# # sgd = SGDClassifier()

# pipe = Pipeline([('vect', vect), ('clf', rfc)])

In [133]:
# Random seach cv

parameters = {
#     'vect__max_df': (0.5),
#     'vect__min_df': (.02),
#     'vect__max_features': (1000),
    'clf__n_estimators':(200, 300),
    'clf__max_depth':(100, 120)
}

rand_search = RandomizedSearchCV(pipe, parameters, cv=5, n_jobs=4, verbose=1)
# grid_search.fit(data.data, data.target)
rand_search.fit(train['description'], train['category'])

C:\Users\Michael\Anaconda3\envs\U4-S1-NLP\lib\site-packages\sklearn\model_selection\_search.py:271: UserWarning: The total space of parameters 4 is smaller than n_iter=10. Running 4 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:   29.8s finished


RandomizedSearchCV(cv=5, error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.7, max_features=1000, min_df=0.02,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=Tru...tors='warn', n_jobs=-1, oob_score=False,
            random_state=42, verbose=0, warm_start=False))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=4,
          param_distributions={'clf__n_estimators': (200, 300), 'clf__max_depth': (100, 120)},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=1)

In [24]:
# no alteration
rand_search.best_score_

0.8503480278422274

In [76]:
rand_search.best_score_

0.8959783449342614

In [77]:
rand_search.best_params_

{'vect__min_df': 0.02,
 'vect__max_features': 1000,
 'vect__max_df': 0.5,
 'clf__n_estimators': 300,
 'clf__max_depth': 100}

In [70]:
rand_search.best_score_

0.8955916473317865

In [71]:
rand_search.best_params_

{'vect__min_df': 0.02,
 'vect__max_features': 800,
 'vect__max_df': 0.7,
 'clf__n_estimators': 200,
 'clf__max_depth': 50}

In [134]:
rand_search.best_score_

0.882830626450116

In [135]:
rand_search.best_params_

{'clf__n_estimators': 300, 'clf__max_depth': 100}

In [205]:
from sklearn.decomposition import TruncatedSVD

In [214]:
vect = TfidfVectorizer(stop_words='english', max_features=1000, ngram_range=(1,2), 
                       min_df=0.02, max_df=.70)
# rfc = RandomForestClassifier()
svd = TruncatedSVD(random_state=42)
sgd = SGDClassifier(max_iter=20)

pipe = Pipeline([('vect', vect), ('svd', svd), ('clf', sgd)])

In [232]:
# SGDClassifier

parameters = {
     'vect__max_df': (.4, 0.5, 0.7),
     'vect__min_df': (.01, .02, .03),
     'vect__max_features': (600, 800, 1000),
#     'clf__n_estimators':(100, 200, 300),
#     'clf__max_depth':(80, 100)
      'clf__max_iter': (10, 20, 50),
      'svd__n_components' : (50, 100, 200),
      'svd__n_iter' : (2, 5, 10, 20)
}

sgd_search = RandomizedSearchCV(pipe, parameters, n_jobs=4, verbose=1)
# grid_search.fit(data.data, data.target)
sgd_search.fit(train['description'], train['category'])

C:\Users\Michael\Anaconda3\envs\U4-S1-NLP\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:   12.6s finished
C:\Users\Michael\Anaconda3\envs\U4-S1-NLP\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
          estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.7, max_features=1000, min_df=0.02,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=Tru...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
          fit_params=None, iid='warn', n_iter=10, n_jobs=4,
          param_distributions={'vect__max_df': (0.4, 0.5, 0.7), 'vect__min_df': (0.01, 0.02, 0.03), 'vect__max_features': (600, 800, 1000), 'clf__max_iter': (10, 20, 50), 'svd__n_components': (50, 100, 200), 'svd__n_iter': (2, 5, 10, 20)},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=1)

In [143]:
sgd.get_params()

{'alpha': 0.0001,
 'average': False,
 'class_weight': None,
 'early_stopping': False,
 'epsilon': 0.1,
 'eta0': 0.0,
 'fit_intercept': True,
 'l1_ratio': 0.15,
 'learning_rate': 'optimal',
 'loss': 'hinge',
 'max_iter': None,
 'n_iter': None,
 'n_iter_no_change': 5,
 'n_jobs': None,
 'penalty': 'l2',
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'tol': None,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [210]:
.get_params

<bound method BaseEstimator.get_params of TruncatedSVD(algorithm='randomized', n_components=5, n_iter=7,
       random_state=42, tol=0.0)>

In [196]:
sgd_search.best_score_

0.9010054137664346

In [197]:
sgd_search.best_params_

{'clf__max_iter': 20}

In [24]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [16]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english')
rfc = RandomForestClassifier()

In [202]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [203]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [13]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission1.csv', index=False)

array([1, 1])

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [25]:
train = df.read_csv('./data/train.csv')
test = df.read_csv('./data/test.csv')

NameError: name 'df' is not defined

## Overview

In [9]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, 
                   algorithm='randomized',
                   n_iter=10)

In [10]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0]
}

In [ ]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

In [ ]:
# Fit
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=4, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  45 out of  45 | elapsed:   23.7s finished
/Users/jonathansokoll/anaconda3/envs/U4-S1-NLP-DS6/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=4,
       param_grid={'lsi__svd__n_components': [10, 100, 250], 'lsi__vect__max_df': [0.9, 0.95, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [ ]:
grid_search.best_score_

0.8786464410735122

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [ ]:
# lsi = ...
# vect = ...
# clf = ...

# pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
# parameters = {
#     'lsi__svd__n_components': [10,100,250],
#     'vect__max_df': (0.75, 1.0),
#     'clf__max_depth':(5,10,15,20)
# }

# grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
# grid_search.fit(..., ...)

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [ ]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission1.csv', index=False)

### Define Pipeline Components

In [ ]:
lsi = ...
vect = ...
clf = ...

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [ ]:
parameters = {
    lsi__svd__n_components': [10,100,250]',
    'vect__max_df': (0.75, 1.0),
    'clf__max_depth':(5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=4, verbose=1)
grid_search.fit(..., ...)

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [ ]:
# Predictions on test sample
pred = pipe.predict(test['description'])

In [ ]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [ ]:
# Make Sure the Category is an Integer
submission.head()

In [ ]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission1.csv', index=False)

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [19]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [20]:
doc = nlp("Two bananas in pyjamas")

In [21]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [26]:
len(X) == len(data.data)

True

In [22]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [23]:
X = get_word_vectors(data.data)

rfc.fit(X, data.target)

/Users/jonathansokoll/anaconda3/envs/U4-S1-NLP-DS6/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [27]:
rfc.score(X, data.target)

0.9918319719953326

## Follow Along

In [27]:
# Apply to your Dataset

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Review

To review this module: 
* Continue working on the Kaggle comeptition
* Find another text classification task to work on